In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [2]:
df_samples = pd.read_excel('12859_2008_2314_MOESM1_ESM.xls', skiprows=2)
# df_samples = df_samples.dropna(how='all', inplace=True).ffill()
df_samples

,Query Accession,Query Description,Reference Accession,Reference Description,CVTree Distance
0,NC_009665.1,Shewanella baltica OS185,NC_008345.1,Shewanella frigidimarina NCIMB 400,0.399260
1,NaN,NaN,NC_008700.1,Shewanella amazonensis SB2B,0.405232
2,NaN,NaN,NC_009052.1,Shewanella baltica OS155,0.145834
3,NaN,NaN,NC_010506.1,Shewanella woodyi ATCC 51908,0.413395
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
742,NaN,NaN,NC_008027.1,Pseudomonas entomophila L48,0.410780
743,NaN,NaN,NC_010501.1,Pseudomonas putida W619,0.408261
744,NaN,NaN,NC_004129.6,Pseudomonas fluorescens Pf-5,0.398769
745,NaN,NaN,NC_007492.1,Pseudomonas fluorescens PfO-1,0.394237


In [3]:
df_positive = pd.read_excel('12859_2008_2314_MOESM2_ESM.xls', skiprows=2)
df_negative = pd.read_excel('12859_2008_2314_MOESM4_ESM.xls', skiprows=2)

In [4]:
def parse_df(df):
    """
    Parse a positive/negative dataset to retrieve th name of the samples
    and the location of their regions of interest (start and stop positions)
    Result is a dictionary {sample_id: [[(start_1,stop_1),...(start_n,stop_n)]}
    """
    res = {}
    df_all = np.split(df, df[df.isnull().all(1)].index) # split by empty rows
    for df_sample in df_all[1:]:
        sample_id = df_sample.iloc[1, 0]
        sample_id, sample_name = sample_id.split(',', 1)

        res[sample_id] = df_sample.iloc[:, 1:3].dropna().astype(int).T.values.tolist()
    return res

In [5]:
res_pos = parse_df(df_positive)
res_neg = parse_df(df_negative)
ground_truth = {sample: {'pos': res_pos[sample], 'neg': res_neg[sample]} for sample in res_pos}

/home/flopau/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/flopau/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
ground_truth

{'NC_009665.1': {'pos': [[642102,
    653689,
    691563,
    722194,
    2237111,
    2332821,
    2354372,
    2492828,
    2501930,
    2607307,
    3031136,
    3042153,
    3860646,
    3888875,
    3973592,
    4002703,
    4735185,
    5040136],
   [650843,
    684682,
    721135,
    733069,
    2246644,
    2342724,
    2368451,
    2501913,
    2514312,
    2624859,
    3039466,
    3052500,
    3883912,
    3898162,
    3996618,
    4014772,
    4747008,
    5054369]],
  'neg': [[245905, 455187, 1745007, 1798225, 1925575, 2990872, 5213182],
   [259554, 467238, 1755025, 1813323, 1934987, 3001784, 5222929]]},
 'NC_007606.1': {'pos': [[406150, 952241, 1748015], [418491, 961243, 1757342]],
  'neg': [[86389,
    109768,
    140708,
    185628,
    284319,
    443427,
    561214,
    620922,
    644057,
    841483,
    1237353,
    1685255,
    1859881,
    2122395,
    2166906,
    2269734,
    2309209,
    2320477,
    2517890,
    2808289,
    2896274,
    2951080,
    3090207,

In [7]:
with open('ground_truth.json', 'w') as f:
    json.dump(ground_truth, f)